In [1]:
import sys
print("PY Directory: " + sys.executable)
# Activate Env
# PS C:\Users\Chickenfish\manimations> uv run python -m ipykernel install --user --name manimations --display-name "Python (manimations)"

import manim as mn
from manim import *
import os, shutil
print("NOT SET? " + shutil.which("latex")) # Should be None
miktex_bin = r"C:\Users\Chickenfish\AppData\Local\Programs\MiKTeX\miktex\bin\x64"
os.environ["PATH"] += ";" + miktex_bin

print("latex  :", shutil.which("latex"))
print("dvisvgm:", shutil.which("dvisvgm"))

config.media_width = "75%"
config.verbosity = "WARNING"

print("Manim Version: "  + mn.__version__)

PY Directory: c:\Users\Chickenfish\manimations\.venv\Scripts\python.exe
NOT SET? C:\Users\Chickenfish\AppData\Local\Programs\MiKTeX\miktex\bin\x64\latex.EXE
latex  : C:\Users\Chickenfish\AppData\Local\Programs\MiKTeX\miktex\bin\x64\latex.EXE
dvisvgm: C:\Users\Chickenfish\AppData\Local\Programs\MiKTeX\miktex\bin\x64\dvisvgm.EXE
Manim Version: 0.19.1


Output Headers

```py
%%manim -qm -o MarketPie_3D --format=mp4 MarketPie
%%manim -qh -o MarketPie_FLAT_SIMPLE --format=png MarketPie
%%manim -qh -o MarketPie_FLAT_SIMPLE --format=mp4 MarketPie
```

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce


file_path = r"../Credit Card Cumulative Section 7 - Ubereats Doordash/data/kikisdelivery.xlsx"

df_door = pd.read_excel(file_path, sheet_name="doordash")
df_uber = pd.read_excel(file_path, sheet_name="ubereats")
df_grub = pd.read_excel(file_path, sheet_name="grubhub")
df_insta = pd.read_excel(file_path, sheet_name="instacart")
df_amazon = pd.read_excel(file_path, sheet_name="amazon fresh")

dfs = [df_door, df_uber, df_grub, df_insta, df_amazon]
names = ["door", "uber", "grubhub", "instacart", "amazon"]

for df, n in zip(dfs, names):
    df.rename(columns={c: f"{c}_{n}" for c in df.columns if c != "YYMM"}, inplace=True)

merged = reduce(lambda left, right: pd.merge(left, right, on="YYMM"), dfs)

yymm = merged["YYMM"].astype(int)
merged["Year"] = (2000 + (yymm // 100)).astype(int)
merged["Month"] = (yymm % 100).astype(int)

def quarter_label(row):
    q = (row["Month"] - 1) // 3 + 1
    return f"{int(row['Year'])} Q{int(q)}"

merged["Quarter"] = merged.apply(quarter_label, axis=1)

quarter_ticks = merged.drop_duplicates(["Year","Quarter"]).index.to_numpy()
quarter_labels = merged.loc[quarter_ticks, "Quarter"].tolist()
x_idx = np.arange(len(merged))

y_door = merged["SPEND_AMOUNT_door"].to_numpy() / 1e6
y_insta = merged["SPEND_AMOUNT_instacart"].to_numpy() / 1e6
y_uber = merged["SPEND_AMOUNT_uber"].to_numpy() / 1e6
y_grub = merged["SPEND_AMOUNT_grubhub"].to_numpy() / 1e6
y_amazon = merged["SPEND_AMOUNT_amazon"].to_numpy() / 1e6


In [ ]:
%%manim -qm --resolution=2560,1440 MonthlySpendingByPlatform
from manim import *
from manim import config

config.frame_width = 22
config.frame_height = 12


class MonthlySpendingByPlatform(Scene):
    def construct(self):
        # Uses globals from previous cell: x_idx, quarter_ticks, quarter_labels,
        # y_door, y_insta, y_uber, y_grub, y_amazon

        max_y = max(
            y_door.max(),
            y_insta.max(),
            y_uber.max(),
            y_grub.max(),
            y_amazon.max(),
        )

        # Axes
        axes = Axes(
            x_range=[0, len(x_idx) - 1, 4],
            y_range=[0, 1.1 * max_y, 50],
            x_length=12,
            y_length=5,
            tips=False,
        ).to_edge(DOWN).shift(UP * 3)

        # Y-axis numeric labels
        y_numbers = axes.get_y_axis().add_numbers()  # <-- add this

        # Title
        title = Text("Monthly Spending by Platform", font_size=36).next_to(axes, UP, buff=1)

        # Axis labels: centered under/along axes
        x_label = Text("Quarter", font_size=30)
        x_label.next_to(axes, DOWN, buff=1)

        y_label = Text("Amount Spent (Millions USD)", font_size=30)
        y_label.rotate(PI / 2)
        y_label.next_to(axes, LEFT, buff=0.5)

        # Quarter tick labels (rotated)
        quarter_label_mobs = VGroup()
        x_axis = axes.get_x_axis()
        for idx, label in zip(quarter_ticks, quarter_labels):
            pos = axes.c2p(idx, 0)
            txt = Text(label, font_size=18).move_to(
                [pos[0], x_axis.get_bottom()[1] - 0.3, 0]
            )
            txt.rotate(PI / 4)
            quarter_label_mobs.add(txt)

        # Manual legend
        legend_items = [
            ("Doordash",     BLUE),
            ("Instacart",    ORANGE),
            ("Ubereats",     GREEN),
            ("Grubhub",      RED),
            ("Amazon Fresh", PURPLE),
        ]

        legend_entries = VGroup()
        for name, color in legend_items:
            sample = Line(ORIGIN, 0.7 * RIGHT, stroke_color=color, stroke_width=6)
            label = Text(name, font_size=20).next_to(sample, RIGHT, buff=0.2)
            entry = VGroup(sample, label)
            legend_entries.add(entry)

        legend_entries.arrange(DOWN, aligned_edge=LEFT, buff=0.15)
        legend_entries.next_to(axes, RIGHT, buff=0.7)

        # Lines built manually from points
        def make_line(x_vals, y_vals, color):
            pts = [axes.c2p(float(x), float(y)) for x, y in zip(x_vals, y_vals)]
            line = VMobject()
            line.set_points_as_corners([*pts]) #nosmooth line.set_points_smoothly(pts)
            line.set_color(color)
            return line

        line_door   = make_line(x_idx, y_door,   BLUE)
        line_insta  = make_line(x_idx, y_insta,  ORANGE)
        line_uber   = make_line(x_idx, y_uber,   GREEN)
        line_grub   = make_line(x_idx, y_grub,   RED)
        line_amazon = make_line(x_idx, y_amazon, PURPLE)

        # Animation
        self.play(FadeIn(title))
        self.play(Create(axes))
        self.play(FadeIn(x_label), FadeIn(y_label), FadeIn(quarter_label_mobs))

        self.play(FadeIn(legend_entries))
        
        # self.play(FadeIn(y_numbers))
        self.play(Create(line_door))
        self.play(Create(line_insta))
        self.play(Create(line_uber))
        self.play(Create(line_grub))
        self.play(Create(line_amazon))

        # Simulate camera backing up: scale whole graph + legend group
        all_stuff = VGroup(
            axes,
            x_label,
            y_label,
            y_numbers, 
            quarter_label_mobs,
            line_door,
            line_insta,
            line_uber,
            line_grub,
            line_amazon,
            legend_entries,
            title,
        )
        self.play(all_stuff.animate.scale(1.1))
        self.wait(2)


Manim Community v0.19.1

In [73]:
import pandas as pd
import re

file_path = r"../Other Useful Data Sheets/bookings-by-brand-USA-2025.xlsx"

# Skip the first 5 rows because they contain titles, not data
df_raw = pd.read_excel(file_path, sheet_name="Data", header=None, skiprows=5)

# Column 1 contains brand names (column B visually)
brands_raw = df_raw[1]

# Column 2 contains strings like "74 in %"
shares_raw = df_raw[2]

# Drop empty rows
brands = brands_raw.dropna().astype(str).tolist()
shares_strings = shares_raw.dropna().astype(str).tolist()

# Extract numeric portion from e.g. "74 in %"
shares = [float(re.findall(r"\d+", s)[0]) for s in shares_strings]

print("BRANDS:", brands)
print("SHARES:", shares)


BRANDS: ['DoorDash', 'Uber Eats', "Domino's", 'GrubHub', 'Pizza Hut', 'KFC', "Papa John's", 'Postmates', 'Delivery.com', 'Caviar', 'ChowNow', 'EatStreet', 'Seamless', 'Slice', 'Other']
SHARES: [74.0, 51.0, 43.0, 30.0, 30.0, 20.0, 20.0, 8.0, 7.0, 5.0, 5.0, 5.0, 3.0, 3.0, 5.0]


In [79]:
%%manim -qm --resolution=2560,1440 DeliveryBrandShare
from manim import *
from manim import config
import numpy as np

config.frame_width = 16
config.frame_height = 9

PASTEL_GREEN = "#77dd77"

class DeliveryBrandShare(Scene):
    def construct(self):
        values = np.array(shares)
        labels = brands
        max_val = float(values.max())

        # Chart
        chart = BarChart(
            values=values,
            bar_names=None,
            y_range=[0, 1.1 * max_val, 10],
            y_length=5,
            x_length=14,
            bar_width=0.40,
            bar_colors=[PASTEL_GREEN] * len(values),
        )
        chart.move_to(ORIGIN)

        # Title
        title = Text(
            "Share of respondents by delivery brand (US, 2025)",
            font_size=36
        ).to_edge(UP)

        # Y-axis label
        y_label = Text("Share of respondents (%)", font_size=26)
        y_label.rotate(PI / 2)
        y_label.next_to(chart, LEFT, buff=0.5)

        # ----------------------------
        # BRAND LABELS (right-aligned, then rotated)
        # ----------------------------
        brand_labels = VGroup()
        for bar, name in zip(chart.bars, labels):
            bar_center = bar.get_center()
            x_axis_y = chart.x_axis.get_bottom()[1]

            # Right-aligned text box
            t = Text(name, font_size=22)
            t.set_alignment("RIGHT")  # <<< KEY: make right edge the anchor

            # Move so the RIGHT EDGE sits at the bar center
            t.move_to([bar_center[0], x_axis_y - 0.35, 0], aligned_edge=RIGHT)

            # Rotate around that aligned right-edge point
            t.rotate(PI/4, about_point=t.get_right())

            # Optional fine-tune shift to pull labels farther away
            t.shift(UP*.25)

            brand_labels.add(t)



        # ----------------------------
        # PERCENTAGE LABELS
        # ----------------------------
        bar_labels = VGroup()
        for bar, v in zip(chart.bars, values):
            lbl = Text(f"{int(v)}%", font_size=24)
            lbl.next_to(bar, UP, buff=0.12)
            bar_labels.add(lbl)

        # ----------------------------
        # GROUPING (so scaling applies IDENTICALLY to everything)
        # ----------------------------
        graph_group = VGroup(chart, y_label, brand_labels, bar_labels)
        graph_group.scale(0.9)
        graph_group.shift(DOWN * 0.4 + RIGHT * 0.2)

        # ----------------------------
        # ANIMATION
        # ----------------------------
        self.play(FadeIn(title))
        self.play(Create(chart.axes))  # only draw axes

        # --- Animate bars + labels together ---
        for idx, (bar, blbl, xlbl) in enumerate(zip(chart.bars, bar_labels, brand_labels)):

            self.play(
                GrowFromEdge(bar, DOWN),
                FadeIn(xlbl, shift=0.2 * DOWN),
                Write(blbl),
                run_time=0.40,
            )

            # Pause after UberEats and before Other
            if idx in (1, len(values) - 2):
                self.wait(0.4)

        self.wait(2)


Manim Community v0.19.1

<string>:49: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().


In [89]:
import pandas as pd

file_path = r"..\Other Useful Data Sheets\food-delivery-revenue-USA.xlsx"

# Load sheet exactly as-is
df_raw = pd.read_excel(file_path, sheet_name="Data", header=None)

# Extract only rows where column 1 (year column) is numeric
data = df_raw[df_raw[1].apply(lambda x: str(x).isdigit())]

# Now extract columns
years   = data[1].astype(int).tolist()
grocery = data[2].astype(float).tolist()
meal    = data[3].astype(float).tolist()

print("Years:", years)
print("Grocery:", grocery)
print("Meal:", meal)


Years: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]
Grocery: [42.8, 50.32, 61.68, 94.15, 126.09, 157.17, 200.97, 257.52, 327.72, 363.8, 393.14, 423.59, 455.18, 488.08]
Meal: [23.55, 28.18, 35.29, 55.91, 70.14, 76.9, 87.8, 96.79, 103.13, 107.38, 110.12, 111.86, 113.15, 114.71]


In [109]:
%%manim -qm --resolution=2560,1440 DeliveryMarketRevenue
from manim import *
from manim import config
import pandas as pd
import numpy as np

config.frame_width = 16
config.frame_height = 9

PASTEL_GREEN = "#77dd77"
PASTEL_PINK  = "#ffb6c1"

class DeliveryMarketRevenue(Scene):
    def construct(self):
        df_raw = pd.read_excel(
            r"..\Other Useful Data Sheets\food-delivery-revenue-USA.xlsx",
            sheet_name="Data",
            header=None
        )

        # keep only numeric-year rows
        data = df_raw[df_raw[1].apply(lambda x: str(x).isdigit())]

        years   = data[1].astype(int).tolist()
        grocery = data[2].astype(float).tolist()
        meal    = data[3].astype(float).tolist()

        n = len(years)
        max_val = max(max(grocery), max(meal))
        y_max = 1.1 * max_val

        # axes: only x-axis visible
        axes = Axes(
            x_range=[-0.5, n - 0.5, 1],
            y_range=[0, y_max, 50],
            x_length=14,
            y_length=5,
            x_axis_config={"color": WHITE, "stroke_width": 2},
            y_axis_config={"stroke_width": 0},
            tips=False,
        ).shift(DOWN * 0.5)

        x_label = Text("Year", font_size=32)
        x_label.next_to(axes.x_axis, DOWN, buff=0.8)

        y_label = Text("Revenue (billion USD)", font_size=28).rotate(PI / 2)
        y_label.next_to(axes.y_axis, LEFT, buff=0.4)

        # bar geometry
        unit_x = axes.c2p(1, 0)[0] - axes.c2p(0, 0)[0]

        # choose width and derive offset so paired bars just touch
        bar_width = 0.25 * unit_x
        x_offset  = bar_width / (2 * unit_x)  # centers separated by exactly bar_width

        grocery_bars = VGroup()
        meal_bars = VGroup()

        for i, (g, m) in enumerate(zip(grocery, meal)):
            # grocery bar
            gb_bot = axes.c2p(i - x_offset, 0)
            gb_top = axes.c2p(i - x_offset, g)
            gb_height = gb_top[1] - gb_bot[1]
            gb_center = (gb_bot + gb_top) / 2

            # meal bar
            mb_bot = axes.c2p(i + x_offset, 0)
            mb_top = axes.c2p(i + x_offset, m)
            mb_height = mb_top[1] - mb_bot[1]
            mb_center = (mb_bot + mb_top) / 2

            gb = Rectangle(
                width=bar_width,
                height=gb_height,
                color=PASTEL_GREEN,
                fill_opacity=1,
                stroke_width=0,
            ).move_to(gb_center)

            mb = Rectangle(
                width=bar_width,
                height=mb_height,
                color=PASTEL_PINK,
                fill_opacity=1,
                stroke_width=0,
            ).move_to(mb_center)

            grocery_bars.add(gb)
            meal_bars.add(mb)

        # year labels
        year_labels = VGroup()
        for i, yr in enumerate(years):
            base = axes.c2p(i, 0)
            t = Text(str(yr), font_size=24)
            t.next_to(base, DOWN, buff=0.3)
            year_labels.add(t)

        # value labels
        g_labels = VGroup()
        m_labels = VGroup()
        for gb, mb, g, m in zip(grocery_bars, meal_bars, grocery, meal):
            g_lbl = Text(f"{g:.2f}", font_size=12).next_to(gb.get_top(), UP, buff=0.06)
            m_lbl = Text(f"{m:.2f}", font_size=12).next_to(mb.get_top(), UP, buff=0.06).shift(RIGHT * 0.125)
            g_labels.add(g_lbl)
            m_labels.add(m_lbl)

        title = Text(
            "Online Food Delivery Market Revenue (US, 2017–2030)",
            font_size=38
        ).to_edge(UP)

        # smaller legend, nudged up and right
        legend = VGroup(
            Square(color=PASTEL_GREEN, fill_opacity=1).scale(0.15),
            Text("Grocery delivery", font_size=22),
            Square(color=PASTEL_PINK, fill_opacity=1).scale(0.15),
            Text("Meal delivery", font_size=22),
        ).arrange(RIGHT, buff=0.35)

        legend.to_corner(UL)
        legend.shift(RIGHT * 1.0 + DOWN * 1.6)

        # animation
        self.play(FadeIn(title))
        self.play(Create(axes.x_axis), FadeIn(x_label), FadeIn(y_label))
        self.play(FadeIn(year_labels), FadeIn(legend))

        for year, gb, mb, g_lbl, m_lbl in zip(
            years, grocery_bars, meal_bars, g_labels, m_labels
        ):
            self.play(
                GrowFromEdge(gb, DOWN),
                GrowFromEdge(mb, DOWN),
                run_time=0.4,
            )
            self.play(FadeIn(g_lbl), FadeIn(m_lbl), run_time=0.2)

            if year == 2025:
                self.wait(3)

        self.wait(2)


Manim Community v0.19.1